In [ ]:
!pip install isbnlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import isbnlib
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans, vq
from pylab import plot, show
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from progressbar import ProgressBar
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")
print('Libraries imported!')

Libraries imported!


In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/ML project/train.csv', error_bad_lines = False)
df

,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label
0,19635,নেভার স্টপ লার্নিং (হার্ডকভার),আয়মান সাদিক,ছাত্রজীবন উন্নয়ন,5,মোটিভেশনাল ভিডিও দেখা শুরু হয়েছে ভাইয়ার ভিডিও ...,Rokomari,positive,2
1,31528,রাসুলুল্লাহ (স) এর নামায (১ম ও ২য় খণ্ড একত্রে...,আল্লামা মুহাম্মদ নাসীরুদ্দীন আলবানী (রহঃ),সালাত/নামায,1,"আসসালামু আলাইকুম।ভাইয়েরা, বইটা নিঃসন্দেহে অবশ্...",Rokomari,negative,0
2,9235,পোয়েটিক জাস্টিস (পেপারব্যাক),আগাথা ক্রিস্টি,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাড...",5,"আগাথা ক্রিস্টির বিখ্যাত ক্রাইম নোভেল ""দ্য মার...",Rokomari,positive,2
3,89993,কমপ্লিট এডোবি ফটোশপ (সিডি সহ) (পেপারব্যাক),বাপ্পি আশরাফ,গ্রাফিক্স ডিজাইন ও মাল্টিমিডিয়া,5,দুর্দান্ত,Rokomari,positive,2
4,155762,মাগফিরাতের পথ ও পাথেয়,ইমাম ইবনু রজব হাম্বলি (রহঃ),আত্মশুদ্ধি ও অনুপ্রেরণা,5,বইঃ মাগফিরাতের পথ ও পাথেয়\r\nলেখক : ইমাম ইবনু ...,WafiLife,positive,2
...,...,...,...,...,...,...,...,...,...
110640,131529,হুমায়ূন আহমেদ : সাহিত্যের জীবন্ত পাঠশালা (হার...,শামীম আহমেদ ১ (সম্পাদক),সাহিত্য ও সাহিত্যিক,5,একটি গন্থ- এ যেন লেখকের নিজের লেখা 'সাহিত্যজীব...,Rokomari,positive,2
110641,155466,কিংবদন্তির কথা বলছি,আহমাদ সাব্বির,ইসলামী ব্যক্তিত্ব,5,ছোটবেলায় আমার আম্মাকে দেখতাম ‘মাসিক মদিনা’ পড়ত...,WafiLife,positive,2
110642,96094,আহকামে যিন্দেগী (পঞ্চম সংস্করণ) (হার্ডকভার),মাওলানা মুহাম্মদ হেমায়েত উদ্দীন,ইসলামি বিধি-বিধান ও মাসআলা-মাসায়েল,5,"বই তা অনেক ভালো,, আমল এর জন্য অনেক ভালো একটা বই",Rokomari,positive,2
110643,11430,কাস্টমার সার্ভিস ১.০ (হার্ডকভার),আনোয়ার এমডি হোসেইন,ক্যারিয়ার উন্নয়ন,5,বাংলাভাষায় কাস্টমার সার্ভিসের উপর খুব বেশি বই...,Rokomari,positive,2


In [ ]:
book_name = df['Book_Name']

book_name
book_name = book_name.apply(lambda x: x.replace(' ', ''))

book_name

0                               নেভারস্টপলার্নিং(হার্ডকভার)
1         রাসুলুল্লাহ(স)এরনামায(১মও২য়খণ্ডএকত্রে)(হার্ডকভার)
2                                পোয়েটিকজাস্টিস(পেপারব্যাক)
3                     কমপ্লিটএডোবিফটোশপ(সিডিসহ)(পেপারব্যাক)
4                                        মাগফিরাতেরপথওপাথেয়
                                ...                        
110640       হুমায়ূনআহমেদ:সাহিত্যেরজীবন্তপাঠশালা(হার্ডকভার)
110641                                    কিংবদন্তিরকথাবলছি
110642              আহকামেযিন্দেগী(পঞ্চমসংস্করণ)(হার্ডকভার)
110643                        কাস্টমারসার্ভিস১.০(হার্ডকভার)
110644                         সন্ধ্যানামারক্ষণে(হার্ডকভার)
Name: Book_Name, Length: 110645, dtype: object

In [ ]:
df['Book_name_without_spaces'] = book_name

df

In [20]:
df = df.drop_duplicates('Book_name_without_spaces')

df

,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces
0,19635,নেভার স্টপ লার্নিং (হার্ডকভার),আয়মান সাদিক,ছাত্রজীবন উন্নয়ন,5,মোটিভেশনাল ভিডিও দেখা শুরু হয়েছে ভাইয়ার ভিডিও ...,Rokomari,positive,2,নেভারস্টপলার্নিং(হার্ডকভার)
1,31528,রাসুলুল্লাহ (স) এর নামায (১ম ও ২য় খণ্ড একত্রে...,আল্লামা মুহাম্মদ নাসীরুদ্দীন আলবানী (রহঃ),সালাত/নামায,1,"আসসালামু আলাইকুম।ভাইয়েরা, বইটা নিঃসন্দেহে অবশ্...",Rokomari,negative,0,রাসুলুল্লাহ(স)এরনামায(১মও২য়খণ্ডএকত্রে)(হার্ডকভার)
2,9235,পোয়েটিক জাস্টিস (পেপারব্যাক),আগাথা ক্রিস্টি,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাড...",5,"আগাথা ক্রিস্টির বিখ্যাত ক্রাইম নোভেল ""দ্য মার...",Rokomari,positive,2,পোয়েটিকজাস্টিস(পেপারব্যাক)
3,89993,কমপ্লিট এডোবি ফটোশপ (সিডি সহ) (পেপারব্যাক),বাপ্পি আশরাফ,গ্রাফিক্স ডিজাইন ও মাল্টিমিডিয়া,5,দুর্দান্ত,Rokomari,positive,2,কমপ্লিটএডোবিফটোশপ(সিডিসহ)(পেপারব্যাক)
4,155762,মাগফিরাতের পথ ও পাথেয়,ইমাম ইবনু রজব হাম্বলি (রহঃ),আত্মশুদ্ধি ও অনুপ্রেরণা,5,বইঃ মাগফিরাতের পথ ও পাথেয়\r\nলেখক : ইমাম ইবনু ...,WafiLife,positive,2,মাগফিরাতেরপথওপাথেয়
...,...,...,...,...,...,...,...,...,...,...
110620,51176,বাংলাদেশের আঞ্চলিক উপন্যাস (হার্ডকভার),গিয়াস শামীম,"প্রবন্ধ, নিবন্ধ ও গবেষণা সমগ্র/সংকলন",5,এটা আসলেই ভালো বই। আঞ্চলিক উপন্যাসগুলোর উপর সু...,Rokomari,positive,2,বাংলাদেশেরআঞ্চলিকউপন্যাস(হার্ডকভার)
110622,142397,নীললোহিতের সেরা ৯ (হার্ডকভার),সুনীল গঙ্গোপাধ্যায় (নীললোহিত),পশ্চিমবঙ্গের বই: উপন্যাস,5,সুনীল গঙ্গোপাধ্যায়ের সবচেয়ে বড় গুণ সে স্পষ্টভা...,Rokomari,positive,2,নীললোহিতেরসেরা৯(হার্ডকভার)
110623,133917,"জিকে মাস্টার (পেপারব্যাক) মেডিকেল, বিশ্ববিদ্...",শিশির আসাদ,সাধারণ জ্ঞান,5,ভাল,Rokomari,positive,2,"জিকেমাস্টার(পেপারব্যাক)মেডিকেল,বিশ্ববিদ্যালয়,ব..."
110627,129184,"চিরসুন্দর লাদাখ (হার্ডকভার) ছবি, রুটম্যাপ এব...",শংকরলাল সরকার,ভারত ভ্রমণ,4,চমৎকার লেগেছে।,Rokomari,positive,2,"চিরসুন্দরলাদাখ(হার্ডকভার)ছবি,রুটম্যাপএবংলাদাখি..."


In [22]:
df = df.sample(n=10000)
df

,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces
53943,111292,ছুটে যাই প্রকৃতির রাজ্যে (হার্ডকভার),মুহাম্মদ জাভেদ হাকিম,বাংলাদেশ ভ্রমণ,4,"বইটা খুব ই উপকারী ভ্রমণ পিপাসুদের জন্য, তবে ছব...",Rokomari,positive,2,ছুটেযাইপ্রকৃতিররাজ্যে(হার্ডকভার)
39338,30087,বিভিন্ন ধম্র্মের উপদেশাবলী (হার্ডকভার),আলহাজ্জ খানবাহাদুর আহ্‌ছানউল্লা,ধর্ম বিষয়ক প্রবন্ধ,5,বইটি আমার জীবনে পড়া সেরা বইয়ের মধ্যে একটি ও অন...,Rokomari,positive,2,বিভিন্নধম্র্মেরউপদেশাবলী(হার্ডকভার)
1424,6850,জি.মাওলা বিপিএসসি নন-টেক প্রিলিমিনারি এনালাইস...,মোঃ খাদিমুল ইসলাম,ইঞ্জিনিয়ারিং নিয়োগ,4,এটা ভাল ছিল।,Rokomari,positive,2,জি.মাওলাবিপিএসসিনন-টেকপ্রিলিমিনারিএনালাইসিস(পে...
14861,6901,হ্যারি পটার অ্যান্ড দ্য প্রিজনার অব আজকাবান (...,জে. কে. রাওলিং,"বয়স যখন ১২-১৭: রহস্য, গোয়েন্দা, থ্রিলার ও অ্...",5,"হ্যারি পটার, একটি বিখ্যাত উপন্যাস।কে হ্যারি পট...",Rokomari,positive,2,হ্যারিপটারঅ্যান্ডদ্যপ্রিজনারঅবআজকাবান(১৯৯৯)(সি...
61083,141371,100 Rules of English Grammar For Any Competit...,Siddique Mohsin Patwary,English Grammar and Language Learning,5,আলো,Rokomari,positive,2,100RulesofEnglishGrammarForAnyCompetitiveExams...
...,...,...,...,...,...,...,...,...,...,...
4969,114048,এস এস সি ম্যাজিক ইংলিশ,মোঃ তানভীরুল হক,English Grammar and Language Learning,5,সেরা একটি বই। তানভীর স্যারের সব বইগুলোই সেরা।,Rokomari,positive,2,এসএসসিম্যাজিকইংলিশ
8074,29965,হিচককের প্রিয় গল্প (হার্ডকভার),আলফ্রেড হিচকক,অনুবাদ গল্প,5,সংগ্রহে রাখার মতন। বেশ ঝরঝরে অনুবাদ। একবার পড়ত...,Rokomari,positive,2,হিচককেরপ্রিয়গল্প(হার্ডকভার)
17,48126,সিংহ ও খরগোশ (পেপারব্যাক),কিডস্‌ বুকস্‌ (সম্পাদক),শিশু-কিশোর গল্প,5,বাচ্চাদের জন্য ভাল চিত্রের গল্পের বই,Rokomari,positive,2,সিংহওখরগোশ(পেপারব্যাক)
98239,123096,ভুতুড়ে পাহাড় ( কিশোর মুসা রবিন সিরিজ ),রকিব হাসান,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও...",5,লেখক রকিব হাসানের কিশোর-মুসা-রবিন ভুতুড়ে পাহাড়...,Rokomari,positive,2,ভুতুড়েপাহাড়(কিশোরমুসারবিনসিরিজ)


In [ ]:
df.reset_index()

df

,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces
53943,111292,ছুটে যাই প্রকৃতির রাজ্যে (হার্ডকভার),মুহাম্মদ জাভেদ হাকিম,বাংলাদেশ ভ্রমণ,4,"বইটা খুব ই উপকারী ভ্রমণ পিপাসুদের জন্য, তবে ছব...",Rokomari,positive,2,ছুটেযাইপ্রকৃতিররাজ্যে(হার্ডকভার)
39338,30087,বিভিন্ন ধম্র্মের উপদেশাবলী (হার্ডকভার),আলহাজ্জ খানবাহাদুর আহ্‌ছানউল্লা,ধর্ম বিষয়ক প্রবন্ধ,5,বইটি আমার জীবনে পড়া সেরা বইয়ের মধ্যে একটি ও অন...,Rokomari,positive,2,বিভিন্নধম্র্মেরউপদেশাবলী(হার্ডকভার)
1424,6850,জি.মাওলা বিপিএসসি নন-টেক প্রিলিমিনারি এনালাইস...,মোঃ খাদিমুল ইসলাম,ইঞ্জিনিয়ারিং নিয়োগ,4,এটা ভাল ছিল।,Rokomari,positive,2,জি.মাওলাবিপিএসসিনন-টেকপ্রিলিমিনারিএনালাইসিস(পে...
14861,6901,হ্যারি পটার অ্যান্ড দ্য প্রিজনার অব আজকাবান (...,জে. কে. রাওলিং,"বয়স যখন ১২-১৭: রহস্য, গোয়েন্দা, থ্রিলার ও অ্...",5,"হ্যারি পটার, একটি বিখ্যাত উপন্যাস।কে হ্যারি পট...",Rokomari,positive,2,হ্যারিপটারঅ্যান্ডদ্যপ্রিজনারঅবআজকাবান(১৯৯৯)(সি...
61083,141371,100 Rules of English Grammar For Any Competit...,Siddique Mohsin Patwary,English Grammar and Language Learning,5,আলো,Rokomari,positive,2,100RulesofEnglishGrammarForAnyCompetitiveExams...
...,...,...,...,...,...,...,...,...,...,...
4969,114048,এস এস সি ম্যাজিক ইংলিশ,মোঃ তানভীরুল হক,English Grammar and Language Learning,5,সেরা একটি বই। তানভীর স্যারের সব বইগুলোই সেরা।,Rokomari,positive,2,এসএসসিম্যাজিকইংলিশ
8074,29965,হিচককের প্রিয় গল্প (হার্ডকভার),আলফ্রেড হিচকক,অনুবাদ গল্প,5,সংগ্রহে রাখার মতন। বেশ ঝরঝরে অনুবাদ। একবার পড়ত...,Rokomari,positive,2,হিচককেরপ্রিয়গল্প(হার্ডকভার)
17,48126,সিংহ ও খরগোশ (পেপারব্যাক),কিডস্‌ বুকস্‌ (সম্পাদক),শিশু-কিশোর গল্প,5,বাচ্চাদের জন্য ভাল চিত্রের গল্পের বই,Rokomari,positive,2,সিংহওখরগোশ(পেপারব্যাক)
98239,123096,ভুতুড়ে পাহাড় ( কিশোর মুসা রবিন সিরিজ ),রকিব হাসান,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও...",5,লেখক রকিব হাসানের কিশোর-মুসা-রবিন ভুতুড়ে পাহাড়...,Rokomari,positive,2,ভুতুড়েপাহাড়(কিশোরমুসারবিনসিরিজ)
